# Model

In [1]:
import os
import sys

import pathlib

notebook_path = pathlib.Path(os.getcwd())
sys.path.append(str(notebook_path.parent))

In [2]:
import numpy as np
import pandas as pd

import pickle

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    PolynomialFeatures
)
from sklearn.pipeline import Pipeline
from sklearn import decomposition
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import (
    cross_val_score,
    GridSearchCV,
    train_test_split
) 
from category_encoders import TargetEncoder

from IPython.display import HTML

from database import engine

## Precalculated tables

Sometimes it's too long to do the calculation at runtime. So in some cases we can prepare pre-computed tables. Here is code to create such tables.

**Note** It is stored in the database in order to be used in production.

TF-IDF is an extremely long operation, but fortunately the post data is constant, so we can pre-compute and store TF-IDF vectors.

## Loading data

- We only load records that belong to the `view` category, because every like starts with view, and we have a `target` column that marks those views that lead to likes.

In [4]:
user_data = pd.read_sql(
    "SELECT * FROM public.user_data;",
    con = engine,
    index_col = "user_id"
)
post_data = pd.read_sql(
    "SELECT * FROM public.kobfedsur_post_features_lesson_22;",
    con = engine,
    index_col = "post_id"
)
post_data = post_data.drop("text" , axis = 1)
feed_data = pd.read_sql(
    """
    SELECT
        timestamp,
        user_id,
        post_id,
        target
    FROM public.feed_data 
    WHERE action='view' 
    LIMIT 200000;
    """,
    con = engine
)

In [5]:
df_show = {
    "Users data" : user_data,
    "Post data" : post_data,
    "Feed data" : feed_data
}

for title, df in df_show.items():
    display(HTML(f"<h3>{title}</h3>"))
    display(df.head())

,gender,age,country,city,exp_group,os,source
user_id,,,,,,,
200,1,34,Russia,Degtyarsk,3,Android,ads
201,0,37,Russia,Abakan,0,Android,ads
202,1,17,Russia,Smolensk,4,Android,ads
203,0,18,Russia,Moscow,1,iOS,ads
204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


,topic,text tf_idf0,text tf_idf1,text tf_idf2,text tf_idf3,text tf_idf4,text tf_idf5,text tf_idf6,text tf_idf7,text tf_idf8,text tf_idf9
post_id,,,,,,,,,,,
1,business,0.386969,-0.178560,-0.090357,-0.092304,-0.124275,0.026649,-0.090906,0.037408,0.021008,0.103114
2,business,0.422381,-0.203720,-0.085703,-0.002275,-0.065920,0.031317,0.018818,0.006605,-0.019381,0.018762
3,business,0.374896,-0.136754,-0.063839,-0.089329,-0.111498,0.020394,-0.058423,0.031442,0.028574,0.055636
4,business,0.258776,-0.118368,-0.052492,-0.053727,-0.096023,0.022188,-0.034058,0.032779,0.016400,0.037955
5,business,0.194117,-0.099521,-0.045276,-0.001843,-0.034211,-0.007644,-0.008759,0.018280,0.013183,0.033272


,timestamp,user_id,post_id,target
0,2021-12-07 20:12:29,71644,1862,0
1,2021-12-07 20:14:56,71644,3948,0
2,2021-12-07 20:17:38,71644,5472,0
3,2021-12-07 20:19:45,71644,702,0
4,2021-12-07 20:21:24,71644,1594,0


## Data preparation

In [6]:
joined_data = pd.merge(
    left = feed_data,
    right = user_data,
    left_on = "user_id",
    right_index = True
)
joined_data = pd.merge(
    left = joined_data,
    right = post_data,
    left_on = "post_id",
    right_index = True
)

In [7]:
X = joined_data.drop(["user_id", "post_id"], axis = 1).copy()

y = X["target"]
X.drop("target", axis = 1, inplace = True)

X["month"] = X["timestamp"].dt.month
X["year"] = X["timestamp"].dt.year
X["hour"] = X["timestamp"].dt.hour
X.drop("timestamp", axis = 1, inplace = True)

X['gender'] = X['gender'].astype("O")
X['exp_group'] = X['exp_group'].astype("O")

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size = 0.8, random_state = 1
)

## Model pipeline

In [9]:
numeric_columns = list(X_train.select_dtypes("number").columns)
categorical_columns = [
    'gender', 'country', 'city', 'os', 'source', 'topic', 'exp_group'
]
cat_nunique = X_train[categorical_columns].nunique()
MTE_columns = cat_nunique.index[cat_nunique > 5].to_list()
OHE_columns = list(set(categorical_columns) - set(MTE_columns))

In [10]:
data_transformer = ColumnTransformer([
    (
        "stand_scaler", StandardScaler(), numeric_columns
    ),
    (
        "one_hot_encod", 
        OneHotEncoder(
            categories = list(X_train[OHE_columns].apply(
                lambda col: list(col.unique()),
                result_type="reduce"
            ))
        ), 
        OHE_columns
    ),
    (
        "mean_target_encoder", 
        TargetEncoder(min_samples_leaf = 0.5, smoothing = 0.5), 
        MTE_columns
    )
])

In [11]:
pipeline = Pipeline([
    ("transfmer", data_transformer),
    ("model", GradientBoostingClassifier())
])

## Model selection

In [14]:
param_grid = {
    "model__learning_rate" : [0.17, 0.2],
    "model__max_depth" : [4],
    "model__n_estimators" : [85, 90, 95]
}

grid_search_result = GridSearchCV(
    estimator = pipeline,
    param_grid = param_grid,
    verbose = 2,
    scoring = "roc_auc",
    return_train_score = True
).fit(X_train, y_train)
pickle.dump(grid_search_result, open("gs_results.pck", "wb"))

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END model__learning_rate=0.17, model__max_depth=4, model__n_estimators=85; total time=  39.2s
[CV] END model__learning_rate=0.17, model__max_depth=4, model__n_estimators=85; total time=  37.4s
[CV] END model__learning_rate=0.17, model__max_depth=4, model__n_estimators=85; total time=  37.2s
[CV] END model__learning_rate=0.17, model__max_depth=4, model__n_estimators=85; total time=  38.6s
[CV] END model__learning_rate=0.17, model__max_depth=4, model__n_estimators=85; total time=  37.0s
[CV] END model__learning_rate=0.17, model__max_depth=4, model__n_estimators=90; total time=  39.8s
[CV] END model__learning_rate=0.17, model__max_depth=4, model__n_estimators=90; total time=  39.4s
[CV] END model__learning_rate=0.17, model__max_depth=4, model__n_estimators=90; total time=  39.1s
[CV] END model__learning_rate=0.17, model__max_depth=4, model__n_estimators=90; total time=  39.3s
[CV] END model__learning_rate=0.17, model__max_de

In [15]:
grid_search_result = pickle.load(open("gs_results.pck", "rb"))
gs_results_frame = pd.DataFrame(grid_search_result.cv_results_["params"])
gs_results_frame["mean_train_score"] = grid_search_result.cv_results_["mean_test_score"]
gs_results_frame["mean_test_score"] = grid_search_result.cv_results_["mean_train_score"]
gs_results_frame.sort_values("mean_train_score", ascending = False)

,model__learning_rate,model__max_depth,model__n_estimators,mean_train_score,mean_test_score
2,0.17,4,95,0.660995,0.706789
3,0.20,4,85,0.660886,0.708734
5,0.20,4,95,0.660827,0.712052
4,0.20,4,90,0.660719,0.710309
0,0.17,4,85,0.660709,0.703446
1,0.17,4,90,0.660690,0.705155


In [16]:
grid_search_result = pickle.load(open("gs_results.pck", "rb"))

best_model = pipeline.set_params(
    **grid_search_result.best_params_
).fit(X_train, y_train)

file_name = "model.pck"
pickle.dump(best_model, open(file_name, "wb"))

## Hitrage@5 estimation

In [17]:
preds_test = pd.Series(
    best_model.predict_proba(X_test)[:,1],
    index = y_test.index
)

data_for_estimation = pd.DataFrame({
    "preds" : preds_test,
    "y" : y_test,
    "user_id" : joined_data.loc[y_test.index, "user_id"]
})

hit_rages = (
    data_for_estimation.
    groupby("user_id").
    apply(lambda x: x.nlargest(5, "preds")["y"].any() if len(x) >= 5 else None)
).dropna().astype("int32")

print("current hitrage@5", hit_rages.mean())

current hitrage@5 0.6236323851203501
